MongoDB is a rich document-oriented NoSQL database. I spent some time diving into it. My philosophy to learn a new tech is always practicing. Therefore, here I share my gained knowledge in the form of the exercise.

Enough of the talk, let’s start!


# Requirement

I am building some personal application with Python. The requirement is to create or update the MongoDB collection(or table in the relational database) with rows in DataFrame. More specifically, for the rows that don't exist we do the creation based on the keys in the DataFrame. Otherwise the update should be performed.



In [1]:
#hide
from pymongo import MongoClient, UpdateOne, collection
from pandas import DataFrame
from numpy import where
from datetime import datetime
    
CONN_STR = 'mongodb+srv://fin-market.x5tm4.mongodb.net/myFirstDatabase?authSource=%24external&authMechanism=MONGODB-X509&tls=true&tlsCertificateKeyFile=../../drivers/X509-cert-fin-market.pem'

In [2]:
def show_collection(collection):
    display(DataFrame([row for row in collection.find()]))

# Setup MongoDB Atlas Database

Get a free cloud based MongoDB database [here](https://www.mongodb.com/cloud/atlas/lp/try2?utm_source=google&utm_campaign=gs_apac_hong_kong_search_core_brand_atlas_desktop&utm_term=mongodb%20atlas&utm_medium=cpc_paid_search&utm_ad=e&utm_ad_campaign_id=12212624344&adgroup=115749709143&gclid=Cj0KCQiAqbyNBhC2ARIsALDwAsApK8irJVZnesxewSitv8kTagWactqhvZPG4gpz5CKx0JV3Fh1pEIsaAttLEALw_wcB).

Connect to the database and create the test database

In [4]:
DB_NAME = 'test'
COLLECTION_NAME = 'employee'

In [16]:
client = MongoClient(CONN_STR)
client.drop_database(DB_NAME)
db = client[DB_NAME] # switch database
collection = db[COLLECTION_NAME] # get the collection

In [17]:
#hide
now = datetime.now()
emplyee = [['user1',25,'male', now],['user2',55,'male', now],['user3',43,'male', now]]
df_emplyee = DataFrame(emplyee, columns=['name','age','sex', 'lastModifiedAt'])
df_emplyee = df_emplyee[['name','age','sex']]

Prepare a test DataFrame df_emplyee that contains three columns
- name
- age
- sex

In [3]:
df_emplyee

,name,age,sex
0,user1,25,male
1,user2,55,male
2,user3,43,male


In [6]:
collection.insert_many(df_emplyee.to_dict("records"))
show_collection(collection)

,_id,name,age,sex
0,61b0d067650e3488de556924,user1,25,male
1,61b0d067650e3488de556925,user2,55,male
2,61b0d067650e3488de556926,user3,43,male


## Update & upsert

Change the age of user1 to 32

In [7]:
myquery = { "name": "user1" }
newvalues = { "$set": { "age": "32" }, "$currentDate": {"lastModifiedAt": { "$type": "date" }} }
collection.update_one(myquery, newvalues)

In [8]:
show_collection(collection)

,_id,name,age,sex,lastModifiedAt
0,61b0d067650e3488de556924,user1,32,male,2021-12-08 15:33:59.537
1,61b0d067650e3488de556925,user2,55,male,NaT
2,61b0d067650e3488de556926,user3,43,male,NaT


In [9]:
collection.update_one({"name":"user1"}, 
                      {"$set":{"age":32},
                              "$currentDate": 
                              {"lastModifiedAt": { "$type": "date" }}
                      }, 
                      upsert=True)
show_collection(collection)

,_id,name,age,sex,lastModifiedAt
0,61b0d067650e3488de556924,user1,32,male,2021-12-08 15:33:59.777
1,61b0d067650e3488de556925,user2,55,male,NaT
2,61b0d067650e3488de556926,user3,43,male,NaT


In [10]:
collection.update_one({"name":"user4"}, 
                      {"$set":{"age":32}, 
                              "$setOnInsert":{"sex":"female"},
                              "$currentDate":{"lastModifiedAt": { "$type": "date" }}
                      }, 
                      upsert=True)
show_collection(collection)

,_id,name,age,sex,lastModifiedAt
0,61b0d067650e3488de556924,user1,32,male,2021-12-08 15:33:59.777
1,61b0d067650e3488de556925,user2,55,male,NaT
2,61b0d067650e3488de556926,user3,43,male,NaT
3,61b0d06712c824e29ed6221a,user4,32,female,2021-12-08 15:33:59.989


# Bulk update

In [11]:
df_emplyee = df_emplyee.append({'name':'user5','age': 65, 'sex':'male'},ignore_index=True)


updates = []
df_emplyee.apply(
        lambda row: updates.append(
            UpdateOne(
                {"name": row.get("name")}, 
                {"$set": row.to_dict(), 
                         "$currentDate":{"lastModifiedAt": { "$type": "date" }}
                }, 
                upsert=True
            )),
        axis=1)
collection.bulk_write(updates)
show_collection(collection)

,_id,name,age,sex,lastModifiedAt
0,61b0d067650e3488de556924,user1,25,male,2021-12-08 15:34:00.236
1,61b0d067650e3488de556925,user2,55,male,2021-12-08 15:34:00.237
2,61b0d067650e3488de556926,user3,43,male,2021-12-08 15:34:00.237
3,61b0d06712c824e29ed6221a,user4,32,female,2021-12-08 15:33:59.989
4,61b0d06812c824e29ed6223b,user5,65,male,2021-12-08 15:34:00.237


# DataFrame upsert

In [12]:
def df_upsert(df:DataFrame, collection, keys:[]):
    def row_query(row, keys ):
        res = {}
        for key in keys:
            res[key] = row.get(key)
        return res
    updates = []
    df_emplyee.apply(
        lambda row: updates.append(
            UpdateOne(
                row_query(row, keys), 
                {'$set': row.to_dict(),
                    "$currentDate":{"lastModifiedAt": { "$type": "date" }}
                }, 
                upsert=True)), 
            axis=1
    )
    collection.bulk_write(updates)


In [13]:
show_collection(collection)

,_id,name,age,sex,lastModifiedAt
0,61b0d067650e3488de556924,user1,25,male,2021-12-08 15:34:00.236
1,61b0d067650e3488de556925,user2,55,male,2021-12-08 15:34:00.237
2,61b0d067650e3488de556926,user3,43,male,2021-12-08 15:34:00.237
3,61b0d06712c824e29ed6221a,user4,32,female,2021-12-08 15:33:59.989
4,61b0d06812c824e29ed6223b,user5,65,male,2021-12-08 15:34:00.237


In [14]:
df_emplyee = df_emplyee.append({'name':'user6','age': 37, 'sex':'female'},ignore_index=True)
df_emplyee

,name,age,sex
0,user1,25,male
1,user2,55,male
2,user3,43,male
3,user5,65,male
4,user6,37,female


In [15]:
df_upsert(df_emplyee, collection, ['name'])
show_collection(collection)

,_id,name,age,sex,lastModifiedAt
0,61b0d067650e3488de556924,user1,25,male,2021-12-08 15:34:00.609
1,61b0d067650e3488de556925,user2,55,male,2021-12-08 15:34:00.610
2,61b0d067650e3488de556926,user3,43,male,2021-12-08 15:34:00.610
3,61b0d06712c824e29ed6221a,user4,32,female,2021-12-08 15:33:59.989
4,61b0d06812c824e29ed6223b,user5,65,male,2021-12-08 15:34:00.610
5,61b0d06812c824e29ed62261,user6,37,female,2021-12-08 15:34:00.610
